In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
import pandas as pd
import numpy as np
# do the LDA
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
import nltk
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
with open('Feature_List', newline='') as f:
    reader = csv.reader(f)
    feature_list = list(reader)

In [3]:
BA_feature=feature_list[0]
DA_feature=feature_list[1]
DS_feature=feature_list[2]
DE_feature=feature_list[3]

In [30]:
all_feature=BA_feature+DA_feature+DS_feature+DE_feature

In [31]:
len(all_feature)

40

In [79]:
len(set(all_feature))

26

In [80]:
clean=["we","our","us","itself","employees","firm","apply","disabilities",
       "re","company","description","accomendation","\nWe","relocation","location"]
stop_words = set(stopwords.words('english')) 
def clean_JD(jd,name):
    name=[x.lower() for x in word_tokenize(name) if x.isalnum()]
    if len(name)>1:
        s=""
        try:
            for i in name:
                s+=i[0]
            name+=[s]
        except:
            pass
    res=""
    for sentence in sent_tokenize(jd):
        for j in sentence.split("\n"):
            if not j: continue
            sent=[]
            for x in word_tokenize(j):
                if x.isalnum():
                    sent.append(x.lower())
            A=set(sent)
            B=set(name+clean)
            if not A&B:
                res+=j+"\n"
    word_tokens = word_tokenize(res)
    wordlist=[word for word in word_tokens if word.isalnum() and word not in stop_words]
    full_text=""
    for word in wordlist:
        full_text+=word+" "
    
    return full_text

In [81]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()

# process the keywords
ba_feature_set = set([ps.stem(tok) for tok in BA_feature]) 
da_feature_set = set([ps.stem(tok) for tok in DA_feature]) 
ds_feature_set = set([ps.stem(tok) for tok in DS_feature]) 
de_feature_set = set([ps.stem(tok) for tok in DE_feature])
all_feature_set = set([ps.stem(tok) for tok in all_feature])

In [103]:
# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    desc=desc.replace("-"," ")
    desc=desc.replace("/"," ")
    tokens = [x.lower() for x in word_tokenize(desc) if x.isalnum()]
    try:tokens.remove("excellent")
    except:pass
    try:tokens.remove("related")
    except:pass
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)

    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD','CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok,tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(tok)for tok in filtered_tokens]
    return set(stemmed_tokens)

In [104]:
def mapping_list(df):
    #name=get_df_name(df)
    #df['details']=name.split("_")[1]
    df['job_description_word_set'] =df['newdetails'].map(prepare_job_desc)
    
    ba_feature = []
    da_feature = []
    ds_feature = []
    de_feature = []
    all_feature = []
   

    num_postings = len(df.index)
    for i in range(num_postings):
        #job_desc = df.iloc[i]['newdetails'].lower()
        job_desc_set = df.iloc[i]['job_description_word_set']

        # check if the keywords are in the job description. Look for exact match by token.
        ba = ba_feature_set.intersection(job_desc_set)
        da = da_feature_set.intersection(job_desc_set)
        ds = ds_feature_set.intersection(job_desc_set)
        de = de_feature_set.intersection(job_desc_set)
        al = all_feature_set.intersection(job_desc_set)

        
        if len(ba) == 0:
            ba=[None]
        if len(da) == 0:
            da=[None]
        if len(ds) == 0:
            ds=[None]
        if len(de) == 0:
            de=[None]
        if len(al) == 0:
            al=[None]
    
                
        ba_feature.append(list(ba))
        da_feature.append(list(da))
        ds_feature.append(list(ds))
        de_feature.append(list(de))
        all_feature.append(list(al))
    
    return ba_feature, da_feature, ds_feature, de_feature,all_feature

In [105]:
def df_with_feature(df):
    ba, da, ds, de, al= mapping_list(df)
    df["ba"]=ba
    df["da"]=da
    df["ds"]=ds
    df["de"]=de
    df["all"]=al
   
    return df

In [106]:
def all(file_name):
    df=pd.read_csv(file_name)[0:5]
    df['newdetails'] =df.apply(lambda x: clean_JD(x['details'],x['company']),axis=1)
    df_with_feature(df)
    return df

In [107]:
all("BA_JD.csv")


,Unnamed: 0,position,company,location,details,newdetails,job_description_word_set,ba,da,ds,de,all
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted by\nAzhar Ali Khan\nTalent Acquisition ...,Posted Azhar Ali Khan Send InMail Business Ana...,"{base, employ, internship, assess, elig, syste...",[None],[None],[None],[data],[data]
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted by\nKhalid Ansari\nRecruitment Speciali...,Posted Khalid Ansari Send InMail Eligibility C...,"{employ, elig, work, graduat, year, post, requ...",[None],[None],[None],[None],[None]
2,2,Business Analyst,Cisco,"New York, NY",Who We Are\n\nCisco Security – at Cisco we del...,Who You Are As Security Licensing Specialist a...,"{present, frequenc, bachelor, interact, track,...",[commun],"[engin, commun]",[None],[engin],"[engin, commun]"
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Job Id: 22461117\n\nThis is a Business Analyti...,Job Id 22461117 The analyst work closely Finan...,"{insight, present, drive, databas, group, work...","[python, learn, commun]","[analyt, program, commun]","[python, learn]","[python, data]","[python, learn, commun, program, data, analyt]"
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Responsibilities vary depending on assigned pr...,Responsibilities vary depending assigned proje...,"{use, present, enhanc, internship, bachelor, t...",[commun],[commun],[None],[None],[commun]


In [108]:
all("DA_JD.csv")

,Unnamed: 0,position,company,location,details,newdetails,job_description_word_set,ba,da,ds,de,all
0,0,Data Analyst,Robert Half,"New York, NY",Description\nPerforming UAT testing of new or ...,Performing UAT testing new changed processes r...,"{program, report, employ, manag, access, world...",[None],[program],[None],[data],"[program, data]"
1,1,Data Analyst - Target Circle (Full-Time Remote...,Target,"Minneapolis, MN",Description\n\nAbout us:\n\nTarget is an iconi...,Behind one world best loved brands uniquely ca...,"{insight, use, team, topic, connecticut, need,...","[python, commun, scientist, spark, sql]","[commun, engin, scientist, spark, analyt, sql]","[python, scientist, statist]","[python, engin, data, statist, spark]","[python, commun, engin, scientist, spark, data..."
2,2,Data Analyst,Dice,"New York, NY",Dice is the leading career destination for tec...,OracleSQL PLSQL Python R Tableau With BSA AML ...,"{aml, python, tableau, experi, servic, year, b...",[python],[None],[python],[python],[python]
3,3,Data Analyst,DistroKid,United States,Summary\n\nThe DistroKid Product team is looki...,Summary The ideal candidate highly motivated r...,"{insight, present, bachelor, launch, manag, te...","[commun, sql]","[analyt, commun, sql]",[None],[data],"[commun, data, analyt, sql]"
4,4,LATAM Markets Data Analyst,Citi,"New York, NY",Job Id: 22460268\n\nThis AVP Data Analyst role...,Job Id 22460268 This AVP Data Analyst role sup...,"{use, locat, avp, task, equiti, team, system, ...",[commun],"[engin, commun, analyt]","[predict, machin, statist]","[engin, data, statist, pipelin]","[predict, machin, commun, pipelin, engin, data..."


In [109]:
all("DS_JD.csv")

,Unnamed: 0,position,company,location,details,newdetails,job_description_word_set,ba,da,ds,de,all
0,0,Data Scientist,CVS Health,"New York, NY","Job Description\n\nDevelops, validates and exe...",Develops validates executes algorithms predict...,"{insight, python, roadblock, educ, structur, b...","[sourc, python, commun, sql]","[engin, analyt, commun, sql]","[python, machin, statist]","[python, pipelin, engin, data, statist]","[sourc, python, machin, commun, pipelin, engin..."
1,1,Part-Time Junior Data Scientist (Remote),"KBR, Inc.","Lexington Park, MD",Junior Data Scientist (Part-Time)\n\nSearching...,Junior Data Scientist Searching data scientist...,"{insight, accur, python, hadoop, scientist, ex...","[python, learn, etl, scientist, sql]","[scientist, sql]","[python, learn, machin, scientist, statist]","[python, etl, data, statist]","[python, learn, machin, etl, scientist, data, ..."
2,2,Data Scientist,CVS Health,"New York, NY","Job Description\n\nAt Aetna, a CVS Health comp...",Position Summary Aetna Clinical Analytics team...,"{insight, structur, team, db2, need, recommend...","[sourc, python, learn, scientist, sql]","[program, scientist, analyt, sql]","[python, learn, machin, scientist, statist]","[python, data, statist, pipelin]","[sourc, python, learn, machin, pipelin, progra..."
3,3,Data Scientist,Intel Corporation,"Atlanta, GA",Job Description\n\nPlatform Security Engineeri...,Platform Security Engineering PSE small team S...,"{ancestri, structur, report, pregnanc, locat, ...",[python],"[engin, program]","[ml, python, machin, statist]","[engin, python, data, statist]","[python, machin, engin, program, data, ml, sta..."
4,4,Part-Time Data Scientist (Remote),"KBR, Inc.","Lexington Park, MD",Searching for a Data Scientist capable of lear...,Searching Data Scientist capable learning inde...,"{insight, python, histori, contribut, scientis...","[python, etl, scientist]",[scientist],"[python, machin, scientist, statist]","[python, etl, data, statist]","[python, machin, etl, scientist, data, statist]"


In [110]:
all("DE_JD.csv")

,Unnamed: 0,position,company,location,details,newdetails,job_description_word_set,ba,da,ds,de,all
0,0,"Data Engineer, 1+ Years of Experience",Snap Inc.,"New York, NY",Snap Inc. is a camera company. We believe that...,What You Do Work closely stakeholders engineer...,"{python, request, comput, eoe, team, principl,...","[python, etl, sql]","[engin, sql]",[python],"[python, pipelin, engin, etl, data, java]","[python, pipelin, engin, etl, data, java, sql]"
1,1,Senior Data Engineer,theSkimm,"New York, NY",theSkimm’\n\nWe’re hiring a Senior Data Engine...,You responsible working alongside analysts pro...,"{insight, report, equiti, team, system, need, ...","[learn, commun, spark]","[commun, engin, program, analyt, collect, spark]","[ml, learn]","[engin, spark, data, pipelin]","[learn, commun, pipelin, engin, program, data,..."
2,2,Sustainable Finance Data Engineer,Morgan Stanley,"New York, NY",The Global Sustainable Finance Group (GSF) aim...,The Global Sustainable Finance Group GSF aims ...,"{gsf, team, system, java, need, capac, group, ...",[None],"[engin, analyt]","[predict, machin]","[engin, data, java]","[predict, machin, engin, data, analyt, java]"
3,3,Data Engineer- Security Analytics & Reporting,IBM,"Armonk, NY",526316BR\n\nIntroduction\n\nAs a Data Scientis...,526316BR Introduction Work Best Class open sou...,"{report, pregnanc, team, system, qualifi, 5263...","[sourc, python, learn, commun, scientist, sql]","[commun, engin, program, scientist, analyt, sql]","[python, learn, scientist, statist]","[python, engin, data, java, statist]","[sourc, python, learn, commun, engin, program,..."
4,4,NEED FOR DATA ENGINEER @ (REMOTE),Dice,United States,Dice is the leading career destination for tec...,JD AWS PAYTHON NAVEEN KUMAR mailto Direct 7329...,"{jd, mailto, 7329672703, aw, kumar}",[None],[None],[None],[None],[None]
